In [6]:
import pandas as pd
import numpy as np
import re
import pymupdf

In [7]:
counties = ["LaGrange"]

In [8]:
i = 0
county = counties[i]
county

'LaGrange'

In [3]:
input_file = "../data-raw/in_2024_results/" + county + ".pdf"
output_file = "../data/in_2024_results/" + county + ".csv"
output_2 = "C:/Users/johnsor/Documents/openelections-data-in/2024/counties/20241105__in__general__" + str.lower(county) + "__precinct.csv"
output_2

'C:/Users/johnsor/Documents/openelections-data-in/2024/counties/20241105__in__general__lagrange__precinct.csv'

In [5]:
doc = pymupdf.open(input_file) # open a document
df = pd.DataFrame({'county' : [], 'precinct' : [], 'office' : [], 'district' : [], 'party' : [], 'candidate' : [], 'votes' : []})
page = doc[0]

NameError: name 'pymupdf' is not defined

In [4]:
blocks = page.get_text_blocks()
df_page = pd.DataFrame(blocks, columns=('tl.x', 'tl.y', 'br.x', 'br.y', 'text', 'index', 'type')) #put the blocks into a dataframe
df_page.sort_values(by=['tl.y'],inplace=True)
df_page

NameError: name 'page' is not defined

In [46]:
def split_cols(page):
    blocks = page.get_text_blocks() #Get the blocks of text
    df_page = pd.DataFrame(blocks, columns=('tl.x', 'tl.y', 'br.x', 'br.y', 'text', 'index', 'type')) #put the blocks into a dataframe
    df_page = df_page[~df_page.text.str.contains(r'Vote For \d')] #Drop the Vote for _ lines
    df_page['text'] = df_page['text'].str.strip() #Strip leading/trailing whitespace from the text
    header = df_page[df_page["br.y"] < 125].sort_values(by=['br.y']) #header at the top
    left_col = df_page[(df_page['br.x'] < 320) & (df_page['br.y'] > 125)].sort_values(by=['br.y']) #left column below the header
    right_col = df_page[(df_page['tl.x'] > 290) & (df_page['br.y'] > 125)].sort_values(by = ['br.y']) #right column below the header
    columns =pd.DataFrame({'text' : pd.concat((left_col.text, right_col.text))}) #Concatenate the sorted columns
                      
    #When No candidate filed, they do not inlclude the zero votes that are needed for what we do next
    columns['text'] = columns.text.str.replace(r'NO CANDIDATE FILED\s+\(([A-Z]+)\)', r'NO CANDIDATE FILED (\1)\n0\n0%', regex=True)
    
    #We need the precinct name, the registered voters, and the total voters from the header
    new = header.text.str.contains(r'Page 1/\d').any()
    precinct = header.text.str.extract(r'Central Count\n(\w+\s*\w*\s*\w*)\s*\d{5,6}').dropna().iloc[0][0]
    reg_voters = int(header.text.str.extract(r'Registered Voters\s+([0-9,.]+)\s+-').dropna().iloc[0][0].replace(',', ''))
    votes = int(header.text.str.extract(r'Total Ballots\s+([0-9,.]+)\s+:').dropna().iloc[0][0].replace(',', ''))
    
    return new, precinct, reg_voters, votes, columns

In [47]:
straight = {'DEMOCRATIC' : 'DEM', 'REPUBLICAN' : 'REP', 'LIBERTARIAN' : 'LIB', 'WE THE PEOPLE' : 'WTP', 'INDIANA REDEMPTION PARTY' : 'RED'}

In [48]:
def cols_to_data(columns):
    tv = columns.text.str.contains('\nTotal Votes') #gives the indices of the Total Votes Lines
    office = tv.shift(-1, fill_value=False) #gives the indices of the office, which is always above Total Votes
    columns['office'] = np.where(office, columns.text, None) #The lines above Total Votes give the office
    columns.fillna(method='ffill', inplace=True) #Forward fill office values
    columns = columns[(~tv) & (~office)] #No longer need the Total Votes lines
    
    #Sometimes the candidates and votes are in the same line, sometimes the votes come on the line before.  To
    # fix this, invert the order, then concatenate by office, then sepaarate by '%'
    columns = columns.iloc[::-1]
    columns = columns.groupby(['office'])['text'].apply(lambda x: ''.join(x)).reset_index()
    columns = columns.assign(text=columns['text'].str.split(r'\n[0-9,.]+%\n*')).explode('text')
    columns = columns[columns['text'] != '']
    columns['text'] = columns['text'].replace({'\s+' : ' '}, regex=True)
    
    #Extract the party and the number of votes
    columns['party'] = columns.text.str.extract(r'\(\s*([A-Z]+)\)\s*\d+$')
    columns['votes'] = columns.text.str.extract(r'(\d+)$')
    
    #To extract the candidates, get rid of either the party and votes or just the votes
    columns['candidate'] = columns.text.str.replace(r'\s*\(\s*([A-Z]+)\)\s*\d+$', '', regex=True)
    columns['candidate'] = columns.candidate.str.replace(r'\s*\d+$', '', regex=True).str.title()
    
    #At this point we've extracted what we needed
    columns = columns.drop('text', axis=1)
    
    #Split the district number off from the office, but only for state senators and federal or state representatives
    columns['district'] = columns['office'].str.extract(r'[SENATOR|REPRESENTATIVE] DISTRICT (\d+)')
    columns['office'] = columns['office'].str.replace(r'(SENATOR|REPRESENTATIVE) DISTRICT (\d+)', r'\1', regex=True).str.title()
    
    return columns

In [49]:
doc = pymupdf.open(input_file) # open a document
df = pd.DataFrame({'county' : [], 'precinct' : [], 'office' : [], 'district' : [], 'party' : [], 'candidate' : [], 'votes' : []})
for page in doc:
    new, precinct, reg_voters, votes, columns = split_cols(page)
    header = pd.DataFrame({'county' : [county]*2, 'precinct' : [precinct]*2, 'office' : ['Ballots Cast','Registered Voters'],
                           'district' : ['',''], 'party' : ['',''], 'candidate' : ['',''], 'votes' : [votes, reg_voters]})
    cols_data = cols_to_data(columns).assign(county = county, precinct=precinct)
    if new:
        df = pd.concat([df, header, cols_data])
    else:
        df = pd.concat([df, cols_data])
#Fix the straight ticket parties
df.loc[df['office'] == 'StraightTicketSelection', 'party'] = df.loc[df['office'] == 'StraightTicketSelection', 'candidate'].map(straight)
df['office'] = df['office'].str.replace('StraightTicketSelection', 'Straight Party')

In [50]:
df

,county,precinct,office,district,party,candidate,votes
0,St. Joseph,CENTRE TWP 1,Ballots Cast,,,,757
1,St. Joseph,CENTRE TWP 1,Registered Voters,,,,1122
0,St. Joseph,CENTRE TWP 1,President Of The United States,NaN,WI,Claudia De La Cruz Karina Garcia,1
0,St. Joseph,CENTRE TWP 1,President Of The United States,NaN,WI,Susan Maude Buchser,0
0,St. Joseph,CENTRE TWP 1,President Of The United States,NaN,WI,Nala Baozun Scott Johnson Jr,0
...,...,...,...,...,...,...,...
9,St. Joseph,WARREN TWP 6,Steele - Judicial Retention,NaN,NaN,No,69
0,St. Joseph,WARREN TWP 6,Telloyan - Judicial Retention,NaN,NaN,Yes,227
0,St. Joseph,WARREN TWP 6,Telloyan - Judicial Retention,NaN,NaN,No,82
1,St. Joseph,WARREN TWP 6,Woods - Judicial Retention,NaN,NaN,Yes,249


In [51]:
df.to_csv(output_file)

In [52]:
df.to_csv(output_2, index=False)